In [3]:
import pandas as pd
import re
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from sklearn.model_selection import train_test_split
import pickle

# Load the dataset
data = pd.read_csv('/content/Customer-Support.csv')

# Data cleaning function
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces and trim
    return text

# Apply data cleaning
data['query'] = data['query'].apply(clean_text)
data['response'] = data['response'].apply(clean_text)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['query'] + data['response'])
query_sequences = tokenizer.texts_to_sequences(data['query'])
response_sequences = tokenizer.texts_to_sequences(data['response'])

# Pad sequences
max_seq_length = max(max(len(seq) for seq in query_sequences), max(len(seq) for seq in response_sequences))
query_padded = pad_sequences(query_sequences, maxlen=max_seq_length, padding='post')
response_padded = pad_sequences(response_sequences, maxlen=max_seq_length, padding='post')

# Split the data
query_train, query_val, response_train, response_val = train_test_split(query_padded, response_padded, test_size=0.2)

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Create the directory if it does not exist
os.makedirs('/mnt/data', exist_ok=True)

# Save tokenizer for later use
with open('/mnt/data/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# Model parameters
embedding_dim = 256
lstm_units = 512

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Print model summary
model.summary()

# Training data
decoder_input_data = np.zeros_like(response_train)
decoder_input_data[:, 1:] = response_train[:, :-1]
decoder_output_data = np.expand_dims(response_train, -1)

# Train the model
model.fit([query_train, decoder_input_data], decoder_output_data, batch_size=64, epochs=20, validation_split=0.2)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            99584     ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 256)            99584     ['input_2[0][0]']             
                                                                                              